# matplotlib.pyplot

## Overview

In [ ]:
# imports
import matplotlib.pyplot as plt

## matplotlib.pyplot example plots


### 1. Scatterplot with histograms

### 2. Scatterplot matrix

### 3. Bubble plot using geographic data

## References